In [1]:
import geopandas as gp
import pandas as pd
import numpy as np
import networkx as nx
import os
from shapely.geometry import Point, Polygon, LineString, mapping
from shapely import geometry
from simpledbf import Dbf5
import warnings
warnings.filterwarnings('ignore')

PyTables is not installed. No support for HDF output.


In [2]:
# GTFS during different times
GTFS_Dir_May2021 = r'C:\Users\xzh263\Dropbox (KTC)\SFCTA CMP\2021 CMP\APC\gtfs_may13'
GTFS_Dir_Apr2020 = r'C:\Users\xzh263\Dropbox (KTC)\SFCTA CMP\2021 CMP\Coverage\gtfs_2020april9'
GTFS_Dir_Oct2019 = r'C:\Users\xzh263\Dropbox (KTC)\SFCTA CMP\2021 CMP\Coverage\gtfs_2019oct9'

# OSM Streets
Streets_Dir = r'Z:\SF_CMP\CMP2021\champ_hwy_shapefile'

# Directory for saving outputs
Coverage_Dir = r'C:\Users\xzh263\Dropbox (KTC)\SFCTA CMP\2021 CMP\Coverage'

In [3]:
#### SFCTA Paths

# GTFS during different times
GTFS_Dir_May2021 = r'Q:\Data\GTFS\MUNI\gtfs_2021may13'
GTFS_Dir_Apr2020 = r'Q:\Data\GTFS\MUNI\gtfs_2020april9'
GTFS_Dir_Oct2019 = r'Q:\Data\GTFS\MUNI\gtfs_2019oct9'

# OSM Streets
Streets_Dir = r'Q:\CMP\LOS Monitoring 2021\Transit\Coverage\champ_hwy_shapefile'

# Directory for saving outputs
Coverage_Dir = r'Q:\CMP\LOS Monitoring 2021\Transit\Coverage'

In [4]:
#Define WGS 1984 coordinate system
wgs84 = {'proj': 'longlat', 'ellps': 'WGS84', 'datum': 'WGS84', 'no_defs': True}

#Define NAD 1983 StatePlane California III
cal3 = {'proj': 'lcc +lat_1=37.06666666666667 +lat_2=38.43333333333333 +lat_0=36.5 +lon_0=-120.5 +x_0=2000000 +y_0=500000.0000000002', 'ellps': 'GRS80', 'datum': 'NAD83', 'no_defs': True}

# GTFS Files 

In [5]:
def generate_transit_stops_geo(stop_dir):
    stops=pd.read_csv(os.path.join(stop_dir, 'stops.txt'))
    stops['geometry'] = list(zip(stops.stop_lon, stops.stop_lat))
    stops['geometry'] = stops['geometry'].apply(Point)
    stops = gp.GeoDataFrame(stops, geometry='geometry', crs={'init': 'epsg:4326'})
    return stops

In [6]:
def generate_transit_shapes_geo(stop_dir):
    shapes=pd.read_csv(os.path.join(stop_dir, 'shapes.txt'))
    shapes_gdf = pd.DataFrame()
    shape_ids = shapes.shape_id.unique().tolist()
    rid = 0
    for shpid in shape_ids:
        shp = shapes[shapes['shape_id']==shpid].sort_values(by='shape_pt_sequence')
        linestr = LineString(zip(shp.shape_pt_lon, shp.shape_pt_lat))
        linestr = gp.GeoDataFrame(index=[shpid], crs='epsg:4326', geometry=[linestr]) 
        shapes_gdf = shapes_gdf.append(linestr)
        rid = rid + 1
    shapes_gdf = shapes_gdf.reset_index()
    shapes_gdf.columns = ['shape_id', 'geometry']
    
    trips = pd.read_csv(os.path.join(stop_dir, 'trips.txt'))
    trips_shapes = shapes_gdf[shapes_gdf['shape_id'].isin(trips['shape_id'])]
    return trips, shapes, trips_shapes

In [7]:
def frequent_bus_routes(gtfs_dir, begin_hour, end_hour, headway, outname):
    routes_info_cur=pd.read_csv(os.path.join(gtfs_dir, 'routes.txt'))
    stops_cur = generate_transit_stops_geo(gtfs_dir)
    trips_cur, shapes_cur, trips_shapes_cur = generate_transit_shapes_geo(gtfs_dir)
    
    #There may be multiples shapes for the same route, so here the most frequent shape is used for each route_id
    trips_shapes_cur_mcv = trips_cur.groupby(['route_id'])['shape_id'].agg(lambda x:x.value_counts().index[0]).reset_index()
    stop_times_cur = pd.read_csv(os.path.join(gtfs_dir, 'stop_times.txt'))
    stop_times_cur['departure_hour'] = stop_times_cur['departure_time'].apply(lambda x: int(x[0:2]))

    trips_cur_hour = pd.merge(trips_cur, 
                          stop_times_cur[stop_times_cur['stop_sequence']==1][['trip_id', 'arrival_time', 'departure_time', 'departure_hour']],
                         on='trip_id', how='left')
    
    trips_cur_period = trips_cur_hour[(trips_cur_hour['departure_hour']>=begin_hour) & (trips_cur_hour['departure_hour']<end_hour)]
    route_hour_counts = trips_cur_period.groupby(['route_id', 'departure_hour']).trip_id.count().reset_index()
    route_hour_counts.columns = ['route_id', 'hour', 'trips']
    frequency = 60/headway
    route_hour_counts_frequent = route_hour_counts[route_hour_counts['trips']>=frequency].groupby(['route_id']).hour.count().reset_index()
    route_hour_counts_frequent.columns = ['route_id', 'hour_count']
    
    num_hours = end_hour - begin_hour
    route_frequent = route_hour_counts_frequent[route_hour_counts_frequent['hour_count']==num_hours]
    route_frequent_shapes = route_frequent.merge(trips_shapes_cur_mcv, on='route_id', how='left')
    
    route_frequent_shapes = trips_shapes_cur.merge(route_frequent_shapes, on='shape_id')
    route_frequent_shapes = route_frequent_shapes.merge(routes_info_cur, on='route_id', how='left')
    route_frequent_shapes.to_file(os.path.join(Coverage_Dir, 'route_frequent_5min_' + outname + '.shp'))
    
    route_frequent_trips = trips_cur_hour[trips_cur_hour['route_id'].isin(route_frequent['route_id'])]
    
    stop_times_by_route = stop_times_cur.merge(trips_cur[['route_id', 'trip_id']], on='trip_id', how='left')
    stop_times_by_route = stop_times_by_route[(stop_times_by_route['departure_hour']>=begin_hour) & (stop_times_by_route['departure_hour']<end_hour)]
    stop_route_hour_count = stop_times_by_route.groupby(['stop_id', 'route_id', 'departure_hour']).trip_id.count().reset_index()
    stop_route_hour_count.columns = ['stop_id', 'route_id', 'hour', 'trips']
    stop_route_hour_count_frequent = stop_route_hour_count[stop_route_hour_count['trips']>=frequency].groupby(['stop_id', 'route_id']).hour.count().reset_index()
    stop_route_hour_count_frequent.columns = ['stop_id', 'route_id', 'hour_count']
    stop_frequent = stop_route_hour_count_frequent[stop_route_hour_count_frequent['hour_count']==num_hours]
    stop_frequent_list = stop_frequent.stop_id.unique().tolist()
    stop_frequent_gdf = stops_cur[stops_cur['stop_id'].isin(stop_frequent_list)]
    stop_frequent_gdf.to_file(os.path.join(Coverage_Dir, 'route_frequent_5min_stops_' + outname + '.shp'))
    
    return stop_frequent_list

In [8]:
# Currently looking at 5-minute headway during 6am - 8pm

# High frequency transit routes May 2021
gtfs_dir, begin_hour, end_hour, headway, outname = GTFS_Dir_May2021, 6, 20, 5, '2021may'
stop_frequent_list_2021may = frequent_bus_routes(gtfs_dir, begin_hour, end_hour, headway, outname)

In [9]:
# High frequency transit routes April 2020
gtfs_dir, begin_hour, end_hour, headway, outname = GTFS_Dir_Apr2020, 6, 20, 5, '2020april'
stop_frequent_list_2020april = frequent_bus_routes(gtfs_dir, begin_hour, end_hour, headway, outname)

In [10]:
# High frequency transit routes October 2019
gtfs_dir, begin_hour, end_hour, headway, outname = GTFS_Dir_Oct2019, 6, 20, 5, '2019october'
stop_frequent_list_2019oct = frequent_bus_routes(gtfs_dir, begin_hour, end_hour, headway, outname)

# OSM Streets

In [11]:
streets = gp.read_file(os.path.join(Streets_Dir, 'champ_freeflow.shp'))
streets.insert(0, 'LinkID', range(1, len(streets)+1))
streets = streets.to_crs(cal3)

In [12]:
def latlong(x):
    return round(x.coords.xy[1][0],6), round(x.coords.xy[0][0], 6), round(x.coords.xy[1][-1], 6), round(x.coords.xy[0][-1], 6)
streets['B_Lat'], streets['B_Long'], streets['E_Lat'], streets['E_Long'] = zip(*streets['geometry'].map(latlong))

b_nodes = streets[['B_Lat', 'B_Long']]
b_nodes.columns = ['Lat', 'Long']

e_nodes = streets[['E_Lat', 'E_Long']]
e_nodes.columns = ['Lat', 'Long']

streets_endnodes = b_nodes.append(e_nodes, ignore_index=True).reset_index()

# Assign unique node id
endnodes_cnt=streets_endnodes.groupby(['Lat', 'Long']).index.count().reset_index()
endnodes_cnt.rename(columns={'index':'NodeCnt'}, inplace=True)
endnodes_cnt['NodeID'] = endnodes_cnt.index+1

# Generate the the unique node shapefile  
endnodes_cnt['geometry'] = list(zip(endnodes_cnt.Long, endnodes_cnt.Lat))
endnodes_cnt['geometry'] = endnodes_cnt['geometry'].apply(Point)
endnodes_unique_gpd = gp.GeoDataFrame(endnodes_cnt, geometry='geometry')
endnodes_unique_gpd.crs = cal3
#endnodes_unique_gpd.to_file(os.path.join(Streets_Dir, 'streets_endnodes.shp'))

In [13]:
endnodes_cnt = endnodes_cnt[['Lat', 'Long', 'NodeCnt', 'NodeID']]
endnodes_cnt.columns = ['B_Lat', 'B_Long', 'B_NodeCnt', 'B_NodeID']

streets = streets.merge(endnodes_cnt, on=['B_Lat', 'B_Long'], how='left')

endnodes_cnt.columns = ['E_Lat', 'E_Long', 'E_NodeCnt', 'E_NodeID']
streets = streets.merge(endnodes_cnt, on=['E_Lat', 'E_Long'], how='left')

endnodes_cnt.columns = ['Lat', 'Long', 'NodeCnt', 'NodeID']

In [14]:
streets['length'] = 3.2808 * streets.geometry.length
streets['b_e'] = list(zip(streets['B_NodeID'], streets['E_NodeID']))
streets['e_b'] = list(zip(streets['E_NodeID'], streets['B_NodeID']))

In [15]:
streets.head(2)

,LinkID,A,B,TOLL,USE,CAP,AT,FT,STREETNAME,TYPE,...,B_Long,E_Lat,E_Long,B_NodeCnt,B_NodeID,E_NodeCnt,E_NodeID,length,b_e,e_b
0,1,6956,40029,0,1,1900,3,2,HWY 101 NORTHBOUND,None,...,1.832939e+06,635754.607976,1.832916e+06,1,51,3,111,799.981191,"(51, 111)","(111, 51)"
1,2,6985,52492,0,1,700,3,4,ALANA,WY,...,1.832827e+06,635823.487134,1.832810e+06,2,82,4,145,547.801408,"(82, 145)","(145, 82)"


In [17]:
streets.to_file(os.path.join(Streets_Dir, 'streets_with_endnodes.shp'))

ValueError: Invalid field type <class 'tuple'>

# Build Walking Network

In [18]:
stops_2021may = generate_transit_stops_geo(GTFS_Dir_May2021)
stops_2021may = stops_2021may.to_crs(cal3)

stops_2020april = generate_transit_stops_geo(GTFS_Dir_Apr2020)
stops_2020april = stops_2020april.to_crs(cal3)

stops = stops_2021may.append(stops_2020april, ignore_index=True)

stops_2019oct = generate_transit_stops_geo(GTFS_Dir_Oct2019)
stops_2019oct = stops_2019oct.to_crs(cal3)

stops = stops.append(stops_2019oct, ignore_index=True)

In [19]:
len(stops)

7381

In [20]:
stops_unique = stops.drop_duplicates(subset=['stop_id']).reset_index()
len(stops_unique)

6053

In [21]:
# find the nearest street link and snap the transit stop on to the link

stops_unique.insert(0, 'NodeID', range(endnodes_cnt['NodeID'].max() + 1, endnodes_cnt['NodeID'].max() + 1 + len(stops_unique)))
import time
start_time0=time.time()

search_radius = 300 # ft

stops_geo = stops_unique.copy()
stops_geo['point_geo'] = stops_geo['geometry']
stops_geo['geometry'] = stops_geo['geometry'].buffer(search_radius/3.2808)
stop_near_links = gp.sjoin(streets[['LinkID', 'B_NodeID', 'E_NodeID', 'length', 'geometry']], stops_geo, op='intersects')

def calc_dist(x):
    stop_point = x['point_geo']
    link_geo = x['geometry']
    x['near_dist'] = stop_point.distance(link_geo)
    x['stop_to_begin'] = link_geo.project(stop_point) * 3.2808  #meters to feet
    x['stop_to_end'] = x['length'] - x['stop_to_begin']
    return x

stop_near_links = stop_near_links.apply(calc_dist, axis=1)
stop_near_links = stop_near_links.sort_values(['stop_id','near_dist'])
stop_near_links = stop_near_links.drop_duplicates('stop_id')
stop_near_links['near_link'] = stop_near_links['LinkID']
stop_near_links['near_link_bid'] = stop_near_links['B_NodeID']
stop_near_links['near_link_eid'] = stop_near_links['E_NodeID']
stop_near_links = stop_near_links.reset_index()

print('Process took %s seconds' %(round(time.time() - start_time0, 1)))
# stop_near_links.to_file(os.path.join(Coverage_Dir, 'transit_stops_nearest_links.shp'))

Process took 123.7 seconds


In [22]:
# construct a network graph
tgraph = nx.Graph() 

# road network nodes
tgraph.add_nodes_from(endnodes_cnt.NodeID.tolist())

# road network links
for i in range (0, len(streets)):
    tgraph.add_edge(streets.loc[i,'B_NodeID'], 
                          streets.loc[i,'E_NodeID'], 
                          weight = streets.loc[i, 'length'])

In [23]:
def walking_area(walk_graph, walk_dis, start_node, link_near_stop):
    cur_path = dict(nx.single_source_dijkstra_path(walk_graph, start_node, cutoff=walk_dis, weight='weight'))
    del cur_path[start_node]
    reach_links = {}
    for key in cur_path:
        sub_path = list(zip(cur_path[key][:-1],cur_path[key][1:]))
        for each_link in sub_path:
            if each_link in reach_links:
                next
            else:
                reach_links[each_link] = 1
    reach_links_df = pd.DataFrame.from_dict(reach_links, orient='index',columns=['accessed']).reset_index()
    reach_links_df.rename(columns={'index':'b_e'},inplace=True)
    streets_access = streets[(streets['b_e'].isin(reach_links_df['b_e'])) | (streets['e_b'].isin(reach_links_df['b_e'])) | (streets['LinkID']==link_near_stop)]
    geom = [x for x in streets_access.geometry]
    multi_line = geometry.MultiLineString(geom)
    multi_line_polygon = multi_line.convex_hull
    
    return multi_line_polygon

In [24]:
# Accessible area from high frequent stops
def frequent_access_area(stop_list, stop_with_nearest_link, buffer_radius):
    start_time0=time.time()
    stop_access_gdf = gp.GeoDataFrame()
    cnt = 0
    for cur_stop_id in stop_list:
        lidx = stop_with_nearest_link.index[stop_with_nearest_link['stop_id']==cur_stop_id][0]
        cur_node_id = stop_with_nearest_link.loc[lidx, 'NodeID']
        cur_link = stop_with_nearest_link.loc[lidx, 'near_link']

        cur_graph = tgraph.copy()
        cur_graph.add_node(cur_node_id)
        cur_graph.add_edge(stop_with_nearest_link.loc[lidx,'near_link_bid'], 
                          stop_with_nearest_link.loc[lidx,'NodeID'], 
                          weight = stop_with_nearest_link.loc[lidx, 'stop_to_begin'])
        cur_graph.add_edge(stop_with_nearest_link.loc[lidx,'NodeID'], 
                              stop_with_nearest_link.loc[lidx,'near_link_eid'], 
                              weight = stop_with_nearest_link.loc[lidx, 'stop_to_end'])

        get_geo = walking_area(cur_graph, buffer_radius, cur_node_id, cur_link)
        cur_access_polygon = gp.GeoDataFrame(index=[0], crs=cal3, geometry=[get_geo])
        stop_access_gdf = stop_access_gdf.append(cur_access_polygon, ignore_index=True)
        cnt = cnt + 1
        if cnt%500==0:
            print('Processed %s Percent, took %s seconds'% ((round(100*cnt/len(stop_list),3)), round(time.time() - start_time0, 1)))
    return stop_access_gdf

In [25]:
buffer_radius = 0.25 * 5280 # a quarter mile walking distance
frequent_stops_access_area_2021may = frequent_access_area(stop_frequent_list_2021may, stop_near_links, buffer_radius)
frequent_stops_access_union_2021may = frequent_stops_access_area_2021may.geometry.unary_union

Processed 49.407 Percent, took 62.7 seconds
Processed 98.814 Percent, took 125.0 seconds


In [26]:
frequent_stops_access_area_2020april = frequent_access_area(stop_frequent_list_2020april, stop_near_links, buffer_radius)
frequent_stops_access_union_2020april = frequent_stops_access_area_2020april.geometry.unary_union

Processed 43.898 Percent, took 62.1 seconds
Processed 87.796 Percent, took 124.6 seconds


In [27]:
frequent_stops_access_area_2019oct = frequent_access_area(stop_frequent_list_2019oct, stop_near_links, buffer_radius)
frequent_stops_access_union_2019oct = frequent_stops_access_area_2019oct.geometry.unary_union

Processed 18.861 Percent, took 62.1 seconds
Processed 37.722 Percent, took 124.3 seconds
Processed 56.582 Percent, took 186.9 seconds
Processed 75.443 Percent, took 249.5 seconds
Processed 94.304 Percent, took 313.1 seconds


# Attach TAZ attributes

In [28]:
taz_shp = gp.read_file(os.path.join(Coverage_Dir, 'TAZ2454_clean\TAZ2454_clean.shp'))
taz_sf_shp = taz_shp[taz_shp['COUNTY']==1] 
print('Number of TAZs in SF', len(taz_sf_shp))
taz_sf_shp = taz_sf_shp.to_crs(cal3)

Number of TAZs in SF 981


In [29]:
taz_dbf = Dbf5(os.path.join(Coverage_Dir, 'tazdata.dbf'))
taz = taz_dbf.to_dataframe()
taz['SFTAZ'] = taz['SFTAZ'].astype(int)
taz_sf = taz_sf_shp.merge(taz, left_on = 'TAZ', right_on = 'SFTAZ', how = 'left')
taz_sf["area_acre"] = taz_sf['geometry'].area * 0.00024711 #Square meters to acres

In [30]:
def frequent_stops_access_taz(frequent_stops_access_union, outname):
    frequent_stops_access_taz= taz_sf_shp['geometry'].intersection(frequent_stops_access_union)

    taz_sf_access_gdf = gp.GeoDataFrame()
    taz_sf_access_gdf['accessarea'] = frequent_stops_access_taz.area* 0.00024711 #Square meters to acres
    taz_sf_access_gdf['index'] = frequent_stops_access_taz.index
    taz_sf_access_gdf['geometry'] = frequent_stops_access_taz.geometry

    taz_sf_access_tazid = taz_sf_access_gdf.merge(taz_sf_shp[['TAZ', 'AREALAND']], left_on='index', right_index=True, how='left')
    taz_sf_access_attrs = taz_sf[['TAZ', 'AREALAND', 'HHLDS', 'TOTALEMP', 'POP', 'area_acre']].merge(taz_sf_access_tazid, on=['TAZ', 'AREALAND'], how='left')
    
    taz_sf_access_attrs['areapcnt'] = 100 * taz_sf_access_attrs['accessarea'] / taz_sf_access_attrs['area_acre']
    taz_sf_access_attrs['access_pop'] = taz_sf_access_attrs['POP'] * taz_sf_access_attrs['areapcnt'] / 100 
    taz_sf_access_attrs['access_jobs'] = taz_sf_access_attrs['TOTALEMP'] * taz_sf_access_attrs['areapcnt'] / 100 
    taz_sf_access_attrs['access_hhlds'] = taz_sf_access_attrs['HHLDS'] * taz_sf_access_attrs['areapcnt'] / 100 
    
    outcols = ['accessarea', 'index', 'TAZ', 'AREALAND', 'HHLDS', 'TOTALEMP',
           'POP', 'area_acre', 'areapcnt', 'access_pop', 'access_jobs', 'access_hhlds']
    #taz_sf_access_attrs[outcols].to_csv(os.path.join(Coverage_Dir, 'frequent_stops_access_taz_intersect_' + outname+ '.csv'), index=False)
    
    return taz_sf_access_attrs[outcols]

In [31]:
frequent_access_taz_attrs_2021may = frequent_stops_access_taz(frequent_stops_access_union_2021may, '2021may')

print('Coverage in 2021 May: ')
print('Percentage of accessible area: ', round(100*frequent_access_taz_attrs_2021may['accessarea'].sum()/frequent_access_taz_attrs_2021may['area_acre'].sum(),2))
print('Percentage of population: ', round(100*frequent_access_taz_attrs_2021may['access_pop'].sum()/frequent_access_taz_attrs_2021may['POP'].sum(),2))
print('Percentage of jobs: ', round(100*frequent_access_taz_attrs_2021may['access_jobs'].sum()/frequent_access_taz_attrs_2021may['TOTALEMP'].sum(),2))
print('Percentage of households: ', round(100*frequent_access_taz_attrs_2021may['access_hhlds'].sum()/frequent_access_taz_attrs_2021may['HHLDS'].sum(),2))

Coverage in 2021 May: 
Percentage of accessible area:  43.28
Percentage of population:  65.04
Percentage of jobs:  80.5
Percentage of households:  68.24


In [32]:
frequent_access_taz_attrs_2020april = frequent_stops_access_taz(frequent_stops_access_union_2020april, '2020april')

print('Coverage in 2020 April: ')
print('Percentage of accessible area: ', round(100*frequent_access_taz_attrs_2020april['accessarea'].sum()/frequent_access_taz_attrs_2020april['area_acre'].sum(),2))
print('Percentage of population: ', round(100*frequent_access_taz_attrs_2020april['access_pop'].sum()/frequent_access_taz_attrs_2020april['POP'].sum(),2))
print('Percentage of jobs: ', round(100*frequent_access_taz_attrs_2020april['access_jobs'].sum()/frequent_access_taz_attrs_2020april['TOTALEMP'].sum(),2))
print('Percentage of households: ', round(100*frequent_access_taz_attrs_2020april['access_hhlds'].sum()/frequent_access_taz_attrs_2020april['HHLDS'].sum(),2))

Coverage in 2020 April: 
Percentage of accessible area:  43.62
Percentage of population:  62.54
Percentage of jobs:  80.22
Percentage of households:  64.64


In [33]:
frequent_access_taz_attrs_2019oct = frequent_stops_access_taz(frequent_stops_access_union_2019oct, '2019oct')
print('Coverage in 2019 October: ')
print('Percentage of accessible area: ', round(100*frequent_access_taz_attrs_2019oct['accessarea'].sum()/frequent_access_taz_attrs_2019oct['area_acre'].sum(),2))
print('Percentage of population: ', round(100*frequent_access_taz_attrs_2019oct['access_pop'].sum()/frequent_access_taz_attrs_2019oct['POP'].sum(),2))
print('Percentage of jobs: ', round(100*frequent_access_taz_attrs_2019oct['access_jobs'].sum()/frequent_access_taz_attrs_2019oct['TOTALEMP'].sum(),2))
print('Percentage of households: ', round(100*frequent_access_taz_attrs_2019oct['access_hhlds'].sum()/frequent_access_taz_attrs_2019oct['HHLDS'].sum(),2))

Coverage in 2019 October: 
Percentage of accessible area:  67.62
Percentage of population:  88.13
Percentage of jobs:  92.72
Percentage of households:  89.05
